In [ ]:
# default_exp lte_01


In [ ]:
# export
import matplotlib.pyplot as plt
plt.ion()
import pathlib
import time
import tqdm
import pandas as pd
import numpy as np
import tqdm
from matplotlib.pyplot import plot, figure, scatter, gca, sca, subplots, subplots_adjust, title, xlabel, ylabel, xlim, ylim, grid
start_time=time.time()


In [ ]:
# export
plt.rcParams["font.size"]=6
plt.rcParams["figure.facecolor"]="w"
plt.rcParams["figure.figsize"]=(10,5,)


In [ ]:
# export
# 30.72 Msps
fs=30_720_000
# extract PUSCH slots
t0=(7.570e-2)
T0=(1.00e-2)
x=np.fromfile("LTE_uplink_847MHz_2022-01-30_30720ksps.sigmf-data", "int16", offset=int(((t0)*(fs)*(4))), count=int(((T0)*(fs)*(2))))
# scale amplitude so that PUSCH QPSK symbols are +/-1, +/-1j
x=((((x[::2])+(((1j)*(x[1::2])))))/((2.00e+4)))
# no carrier at DC, central carriers at +/-7.5kHz
# shift up by 5.94kHz to move left central subcarrier to DC. 
# carrier frequency offset measured with PUSCH and DMRS to be ~1.5kHz.
# phase offset with DMRS signal
delta_f=(5.940e+3)
delta_phi=(1.960    )
x=((x)*(np.exp(((1j)*(((((2)*(np.pi)*(((delta_f)/(fs)))*(np.arange(x.size))))+(delta_phi)))))))
print("{:7.6f} \033[31m carrier frequency offset \033[0m (- delta_f 7500.0)={}".format(((time.time())-(start_time)), ((delta_f)-((7.50e+3)))))
Tu=2048
print("{:7.6f} \033[31m useful time of a symbol \033[0m Tu={}".format(((time.time())-(start_time)), Tu))
cp_len=144
print("{:7.6f} \033[31m 4.7usec, used in symbols 1-6 in a slot \033[0m cp_len={}".format(((time.time())-(start_time)), cp_len))
cp_len2=160
print("{:7.6f} \033[31m 5.2usec, used in symbol 0 in a slot \033[0m cp_len2={}".format(((time.time())-(start_time)), cp_len2))


In [ ]:
# export
cp_corr=np.empty(((x.size)-(cp_len)-(Tu)), np.complex64)
for j in tqdm.tqdm(range(cp_corr.size)):
    cp_corr[j]=np.sum(((x[j:][:cp_len])*(np.conjugate(x[((j)+(Tu)):][:cp_len]))))
print("{:7.6f} \033[31m correlate the end of each symbol with its own cyclic prefix \033[0m ".format(((time.time())-(start_time))))
L=((20)*(Tu))
xsample=np.arange(L)
xtime=(((1.00e+6))*(((xsample)/(fs))))
print("{:7.6f} \033[31m  \033[0m (len cp_corr)={} L={}".format(((time.time())-(start_time)), len(cp_corr), L))
fig, axs=subplots(4, 1, figsize=[12, 17])
subplots_adjust(left=(0.10    ), right=(0.90    ), bottom=(5.00e-2), top=(0.950    ), wspace=(0.30    ), hspace=(0.40    ))
ax=axs[0]
plt.sca(ax)
plot(xsample, np.real(cp_corr[:L]), label="I")
# upper axis function of lower axis
# https://stackoverflow.com/questions/10514315/how-to-add-a-second-x-axis-in-matplotlib
ax2=ax.twiny()
ax2.set_xticks(ax.get_xticks())
ax2.set_xbound(ax.get_xbound())
ax2.set_xticklabels([(((1.00e+6))*(((x)/(fs)))) for x in ax.get_xticks()])
ax2.set_xlabel("time (us)")
ax.set_xlabel("time (sample)")
ylabel("I")
grid()
plt.legend()
ax=axs[1]
plt.sca(ax)
plot(xsample, np.imag(cp_corr[:L]), label="Q")
# upper axis function of lower axis
# https://stackoverflow.com/questions/10514315/how-to-add-a-second-x-axis-in-matplotlib
ax2=ax.twiny()
ax2.set_xticks(ax.get_xticks())
ax2.set_xbound(ax.get_xbound())
ax2.set_xticklabels([(((1.00e+6))*(((x)/(fs)))) for x in ax.get_xticks()])
ax2.set_xlabel("time (us)")
ax.set_xlabel("time (sample)")
ylabel("Q")
grid()
plt.legend()
ax=axs[2]
plt.sca(ax)
plot(xsample, np.abs(cp_corr[:L]), label="abs")
# upper axis function of lower axis
# https://stackoverflow.com/questions/10514315/how-to-add-a-second-x-axis-in-matplotlib
ax2=ax.twiny()
ax2.set_xticks(ax.get_xticks())
ax2.set_xbound(ax.get_xbound())
ax2.set_xticklabels([(((1.00e+6))*(((x)/(fs)))) for x in ax.get_xticks()])
ax2.set_xlabel("time (us)")
ax.set_xlabel("time (sample)")
ylabel("abs")
grid()
plt.legend()
ax=axs[3]
plt.sca(ax)
plot(xsample, np.angle(cp_corr[:L]), label="angle")
# upper axis function of lower axis
# https://stackoverflow.com/questions/10514315/how-to-add-a-second-x-axis-in-matplotlib
ax2=ax.twiny()
ax2.set_xticks(ax.get_xticks())
ax2.set_xbound(ax.get_xbound())
ax2.set_xticklabels([(((1.00e+6))*(((x)/(fs)))) for x in ax.get_xticks()])
ax2.set_xlabel("time (us)")
ax.set_xlabel("time (sample)")
ylabel("angle")
grid()
plt.legend()
